### wls_estimator.ipynb
- Siep Dokter
- Emil Jousimaa
- Oleksandr Sosovskyy
- Mario Gabriele Carofano

> This file contains the implementation of the WLS estimator, used for estimate the target position starting from the RSS information coming from the anchors.

In [127]:
# IMPORTS
import import_ipynb
import constants
import auxfunc
import pandas as pd
import numpy as np
import pprint
import math

In [128]:
def calculate_target_anchors_distance(anchor_dataframe):
    """
    Calculates the distance between the target and the anchors using the Euclidean norm.
    This is the implementation of (1) on the reference paper.

    Parameters:
    anchor_dataframe (DataFrame): A DataFrame-object containing the dataset of one anchor.

    Returns:
    target_anchors_distance (list): Returns a list of distances between the target and the anchor.
    """

    anchor_coords = [eval(i) for i in anchor_dataframe["Relative Coordinates [m]"][0].split(", ")]
    target_coords = [eval(i) for i in anchor_dataframe["Target Coordinates [m]"][0].split(", ")]
    target_anchors_distance = math.dist(anchor_coords, target_coords)

    return target_anchors_distance

# https://www.w3schools.com/python/ref_math_dist.asp
# https://www.w3schools.com/python/ref_string_split.asp
# https://www.geeksforgeeks.org/python-converting-all-strings-in-list-to-integers/

In [129]:
def calculate_target_anchors_estimation(mean_values):
    """
    Calculates the distance estimates between each anchor and the target.
    This is the implementation of (4) on the reference paper.

    Parameters:
    mean_values (list): A list which values are mean values over the given amount of bursts.

    Returns:
    estimated_distances (list): Returns a list which values are the distance estimations.
    """
    
    estimated_distances = []
    index = 0
    
    for mean in mean_values:
        num = mean - constants.REFERENCE_POWER
        den = -10 * constants.PATH_LOSS_EXPONENT
        estimated_distances.append(round(10**(num/den), 3))
        index = index + 1

    return estimated_distances

In [130]:
def calculate_A_matrix(n_anchors, configuration):
    local_anchor_coords = configuration["Coordinates"]

    # Build matrix A
    A = np.zeros((n_anchors, 3))  # 3 columns for -2x, -2y, 1

    for i in range(n_anchors):
        A[i, 0] = -2 * local_anchor_coords[i][0]
        A[i, 1] = -2 * local_anchor_coords[i][1]
        A[i, 2] = 1

    return A

In [131]:
def calculate_b_vector(n_anchors, configuration):
    """
    Calculates the 'b' vector of the system of equations which the WLS method solves.
    This is the implementation of (5) on the reference paper.

    Parameters:
    n_anchors (int): An integer which value represents the number of anchors in dataset.
    configuration (dict): A dict which keys are the anchor names and values are the respective configurations.

    Returns:
    b (numpy.array): Returns a 1D numpy.array which values are elements of the 'b' vector for the selected configuration.
    """
    
    # A list containing one real number for each anchor.
    # That value is the distance estimation between each anchor and the target.
    local_estimated_distances = configuration["Estimated Distance"]
    
    # A list containing one list for each anchor.
    # Those lists are the 2D-coordinates of each anchor.
    local_anchor_coords = configuration["Coordinates"]
    norm_coords = []

    b = []

    for i in range(n_anchors):
        # print(local_estimated_distances[i])
        sqr_distance = math.pow(local_estimated_distances[i], 2)
        norm_coords.append(np.linalg.norm(local_anchor_coords))
        sqr_coords = math.pow(norm_coords[i], 2)
        diff = sqr_distance - sqr_coords
        b.append(round(diff, 3))
        i = i + 1
    
    return np.array(b)

# https://www.digitalocean.com/community/tutorials/norm-of-vector-python
# https://numpy.org/doc/stable/reference/generated/numpy.append.html

In [132]:
def calculate_W_matrix(n_anchors, configuration):
    """
    Calculates, for the selected configuration, the 'W' diagonal matrix.
    The elements are the inverse of the variance of the square of the estimated distance.
    This is the implementation of [11, eq. (14)].

    Parameters:
    n_anchors (int): An integer which value represents the number of anchors in dataset.
    configuration (dict): A dict which keys are the anchor names and values are the respective configurations.

    Returns:
    W (numpy.array): Returns a 2D numpy.array which values are elements of the 'W' diagonal matrix for the selected configuration.
    """

    # A list containing one real number for each anchor.
    # That value is the distance estimation between each anchor and the target.
    local_estimated_distances = configuration["Estimated Distance"]

    standard_deviation = 2
    num = math.pow(standard_deviation, 2)
    den = 4.715 * math.pow(constants.PATH_LOSS_EXPONENT, 2)
    exp_value = math.exp(num/den)
    var_inverse = []

    for i in range(n_anchors):
        e4_distance = math.pow(local_estimated_distances[i], 4)
        var = e4_distance * exp_value * (exp_value-1)
        var_inverse.append(round(math.pow(var, -1), 3))

    return np.diag(var_inverse)
    
# https://www.w3schools.com/python/ref_math_exp.asp
# https://numpy.org/doc/stable/reference/generated/numpy.diag.html

In [133]:
def calculate_WLS_output(A, W, b):
    """
    Calculate the Weighted Least Squares (WLS) position estimate.
    This is the implementation of (6) on the reference paper.

    Parameters:
    A (numpy.array): Matrix A.
    W (numpy.array): Weight matrix W.
    b (numpy.array): Vector b.

    Returns:
    position (numpy.array): WLS position estimate.
    """
    
    A_transpose = np.transpose(A)
    
    # Calculate (A^T * W * A)^(-1) * A^T * W * b
    left_term = np.linalg.inv(np.matmul(np.matmul(A_transpose, W), A))
    right_term = np.matmul(np.matmul(A_transpose, W), b)

    # Calculate the final result
    position = np.matmul(left_term, right_term)

    return position

# https://numpy.org/doc/stable/reference/generated/numpy.matmul.html

In [134]:
def calculate_target_position():
    directory_path = constants.SCENARIO_A_PATH + "/RPI/RSS_BLT_Dataset/"
    anchors = ["Anchor 1", "Anchor 2", "Anchor 3", "Anchor 4", "Anchor 5", "Anchor 6"]
    dataframes = auxfunc.read_data(anchors, directory_path)
    length = auxfunc.calculate_smallest_dataset(dataframes)
    
    burst_quantity = auxfunc.calculate_burst_quantity(length)
    n_anchors = len(anchors)

    mean_values = {}
    estimated_distances = {}
    configurations = {}
    target_position = {}
    
    for index in range(burst_quantity):
        configurations[index] = {}
        configurations[index]["Mean Power"] = []
        configurations[index]["Estimated Distance"] = []
        configurations[index]["Actual Distance"] = []
        configurations[index]["Coordinates"] = []
        for anchor in dataframes:
            mean_values[anchor] = auxfunc.calculate_average_RSS(dataframes[anchor], length)
            estimated_distances[anchor] = calculate_target_anchors_estimation(mean_values[anchor])
            configurations[index]["Mean Power"].append(mean_values[anchor][index])
            configurations[index]["Estimated Distance"].append(estimated_distances[anchor][index])
            configurations[index]["Actual Distance"].append(dataframes[anchor]["Distance Target - Anchor [m]"][0])
            configurations[index]["Coordinates"].append([eval(i) for i in dataframes[anchor]["Relative Coordinates [m]"][0].split(", ")])
    
    for index in range(burst_quantity):
        configurations[index]["A matrix"] = []
        configurations[index]["b vector"] = []
        configurations[index]["W matrix"] = []
    
        A_mat = calculate_A_matrix(n_anchors, configurations[index])
        B_vet = calculate_b_vector(n_anchors, configurations[index])
        W_mat = calculate_W_matrix(n_anchors, configurations[index])
                             
        configurations[index]["A matrix"].append(A_mat)
        configurations[index]["b vector"].append(B_vet)
        configurations[index]["W matrix"].append(W_mat)

        target_position[index] = calculate_WLS_output(A_mat, W_mat, B_vet)
    
    # np.set_printoptions(suppress=True)
    # pprint.pprint(configurations)

    return target_position
    
# https://stackoverflow.com/questions/9777783/suppress-scientific-notation-in-numpy-when-creating-array-from-nested-list

In [135]:
target_estimation = calculate_target_position()
np.set_printoptions(suppress=True)
pprint.pprint(target_estimation)

{0: array([   -1.11055074,    -0.03960348, -1146.27733798]),
 1: array([   -1.18711821,    -0.04997516, -1143.99649484]),
 2: array([   -0.7727409 ,    -0.25751039, -1149.89778737]),
 3: array([   -0.99289906,     0.03625488, -1146.30676309]),
 4: array([   -0.90728404,    -0.03561628, -1149.150085  ]),
 5: array([   -0.93444289,    -0.02238898, -1149.33331336]),
 6: array([   -0.90606926,    -0.1960183 , -1147.88247084]),
 7: array([   -0.91510742,    -0.15425941, -1148.46414513]),
 8: array([   -0.86039009,    -0.05991864, -1151.38763704]),
 9: array([   -0.78153811,    -0.17356198, -1150.58432196]),
 10: array([   -0.85230656,    -0.10443902, -1151.08828799]),
 11: array([   -0.77535963,    -0.06238625, -1152.81769728]),
 12: array([   -0.83796843,    -0.12349142, -1150.4319526 ]),
 13: array([   -0.73503915,    -0.13088926, -1151.15122855]),
 14: array([   -0.84631877,    -0.27298949, -1148.0668972 ]),
 15: array([   -0.82230763,    -0.1980834 , -1148.55248951]),
 16: array([   -0.

### References
[https://stackoverflow.com/questions/20186344/importing-an-ipynb-file-from-another-ipynb-file](https://stackoverflow.com/questions/20186344/importing-an-ipynb-file-from-another-ipynb-file)